In [9]:
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage

workflow = StateGraph(MessagesState)

llm = Ollama(model="llama2")

llm.invoke([
    HumanMessage("hello, I am Sifat"),
])

"\nAssistant: Hello Sifat! It's nice to meet you. How can I assist you today? Is there something you would like to talk about or ask?"

In [17]:
from langchain_community.llms import Ollama
from langchain.schema import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph

# -------------------------
# Workflow definition
# -------------------------
workflow = StateGraph(dict)  # Using plain dict as state

# -------------------------
# Model setup
# -------------------------
llm = Ollama(model="llama2")  # Make sure you have pulled this model

# -------------------------
# Node: call LLM
# -------------------------
def call_model(state: dict):
    # The LLM expects a list of messages
    messages = state.get("messages", [])
    response = llm.invoke(messages)

    # Append assistant response to history
    messages.append({"role": "assistant", "content": response})
    state["messages"] = messages
    state["response"] = response
    return state

# -------------------------
# Add node and edge correctly
# -------------------------
workflow.add_node("chat_model", call_model)  # Add node only once
workflow.add_edge(START, "chat_model")       # Add edge after node exists

# -------------------------
# Memory saver
# -------------------------
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# -------------------------
# Query function
# -------------------------
def ask_question(query: str, thread_id: str = "session_1"):

    config = {"configurable": {"thread_id": thread_id}}
    result = app.invoke(input=state, config=config)

    # Save updated state to memory automatically happens via checkpointer
    return result["response"]

# -------------------------
# Run chat loop
# -------------------------
if __name__ == "__main__":
    while True:
        try:
            message = input("Your message: ")
        except EOFError:
            print("\nExiting chat.")
            break
        if not message.strip():
            continue

        answer = ask_question(message)
        print("\nResponse:\n", answer)


AttributeError: 'InMemorySaver' object has no attribute 'load'